In [1]:
import MagmaPandas.volatile_solubility.models.IaconoMarziano as im
import MagmaPandas.volatile_solubility.models.Allison2022 as a
import MagmaPandas.geochemistry.volatile_solubility_models.shiskina as sh
sh_config = sh.shiskina_configuration
sh_config.print()
# print("\n")
# a_config = a.Allison_configuration
# a_config.print()

##### Shiskina volatile solubility #####
########################################
Settings________________________________
Fugacity model.....................ideal

Calibration range______________________
Temperature..................1423-1523°K
Pressure......................0.2-5 kbar




In [2]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.parse_io.readers as r
import pandas as pd
import elements as e
import numpy as np
import matplotlib.pyplot as plt
import VESIcal as v
config = mp.configuration
print(config)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................1
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed


/Users/thomas/miniforge3/lib/python3.9/site-packages/VESIcal/calculate_classes.py:7: UserWarning: 

  from VESIcal.models import magmasat


In [3]:

melt = mp.read_melt('./melt.csv', index_col=['name'], total_col='total')

In [4]:
from MagmaPandas.geochemistry.volatiles import _get_solubility_model

_get_solubility_model("IaconoMarziano", species="mixed")

MagmaPandas.geochemistry.volatile_solubility_models.IaconoMarziano.mixed

In [5]:
melt.volatile_saturation_pressure(1400)

|██████████████████████████████| 29/29 [100%] in 1.6s (17.84/s)                                                         16/29 [55%] in 1s (14.5/s, eta: 2s) 


name
PI032-01-01    3008.745618
PI032-01-04    2487.147315
PI032-02-01    3215.947608
PI032-02-02    3007.794701
PI032-02-03    3091.293670
PI032-03-01    2657.385018
PI032-04-01    4095.271425
PI032-04-02    4226.660253
PI032-04-03     745.823944
PI032-05-01    2704.144390
PI041-02-01    3355.798090
PI041-02-02    1955.061585
PI041-03-01    3543.259629
PI041-03-03    3493.129849
PI041-04-02    3343.534704
PI041-04-03    3384.101320
PI041-04-04    3336.702607
PI041-04-05    3462.929643
PI041-04-06    3571.906083
PI041-04-07    3612.379636
PI041-05-02    1932.870396
PI041-05-03    1992.110660
PI041-05-04    1387.262907
PI041-05-06    2513.329301
PI041-07-01    3665.291541
PI041-07-02    2447.695777
PI041-07-06    3132.622075
PI052-01-02    1403.966322
PI052-02-01    2075.370310
dtype: float64

In [6]:
sample = "PI032-04-01"
FeO_initial = 15
Fo_host = 0.6
Kd = 0.29
P_bar = 1e3
T_K = 1500
inclusion = melt.loc[sample,:].copy()
# inclusion["H2O"] = 5


In [7]:
import MagmaPandas.volatile_solubility as vs
vs.calculate_saturation(inclusion, 1400)

4095.271424982669

In [ ]:
s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
s.get_composition()

In [ ]:
v.get_model_names()

In [ ]:
print(f"{'Sample': <15}{'VESIcal':<10}{'Me':<10}{'delta': <7}{'X-H2O': >}")
for idx in range(20):

    inclusion = melt.iloc[idx,:]
    s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
    model = "IaconoMarziano"
    VC = v.calculate_saturation_pressure(s, temperature=T_K-273, model=model).result
    IM, xIM = im.calculate_saturation(inclusion, T_K, output="both")
    # A, xA = a.calculate_saturation(inclusion, T_K, output="both")
    # SH, x = sh.mixed.calculate_saturation(inclusion, output="both")
    # string = f"IM: {IM:>5.0f}, A: {A:>5.0f}"    
    print(f"{inclusion.name:<15}{VC:<10.0f}{IM:<10.0f}{(VC - IM): <7.0f}{xIM: >.2f}")
    # print(f"VESIcal: {VC:<10.0f} {string:>25}, delta: {(VC - IM): >3.0f} bar, Xfl-IM: {xim: >4.2f}, Xfl-A: {x: .2f}")

In [ ]:
model = "IaconoMarzianoWater"

p = np.arange(1,  1e4, 5e1)
fig, ax = plt.subplots(figsize=(10, 10))

plt.plot(p, [im.h2o.calculate_solubility(inclusion, P_bar=i, T_K=T_K) for i in p], label="Me", linewidth=4, alpha=0.8, linestyle="--")
plt.plot(p, [v.calculate_dissolved_volatiles(s, temperature=T_K-273.15, pressure=i, model=model).result for i in p], label="VESIcal")

plt.legend()
plt.show()

In [ ]:
melt.iloc[0, :]